In [ ]:
import numpy as np
import pprint as pp     # pretty printing module
from matplotlib import pyplot as plt        # required only for plotting results
from mondrianforest_utils import load_data, reset_random_seed, precompute_minimal 
from mondrianforest import process_command_line, MondrianForest

In [ ]:
# ./mondrianforest_demo.py --dataset satimage --n_mondrians 100 --budget -1 
#    --normalize_features 1 --save 1 --data_path ../process_data/ --n_minibatches 10 
#    --store_every 1 --optype class

class settings:
    alpha = 0
    bagging = 0
    budget= -1.0
    budget_to_use= float("inf")
    data_path = 'C:\\Users\\kkenda\\Desktop\\moa\\moa\\src\\examples\\java\\moa\\experiments\\data/'
    dataset = 'id12041022_1_AR_WF_DT.arff'
    debug = 0
    discount_factor = 10
    draw_mondrian = 0
    init_id = 1
    min_samples_split = 2
    n_minibatches = 4000
    n_mondrians = 10
    name_metric = 'mse'
    normalize_features = 1
    op_dir = 'results'
    optype = 'real'
    perf_dataset_keys = ['train', 'test']
    perf_metrics_keys = ['log_prob', 'acc']
    perf_store_keys = ['pred_prob']
    save = 1
    select_features = 0
    smooth_hierarchically = 0
    store_every = 0
    tag = ''
    verbose = 1
  

In [ ]:
data = load_data(settings)

In [ ]:
param, cache = precompute_minimal(data, settings)
mf = MondrianForest(settings, data)


In [ ]:
print '\nminibatch\tmetric_train\tmetric_test\tnum_leaves'

#for idx_minibatch in range(settings.n_minibatches):
for idx_minibatch in range(50):
    train_ids_current_minibatch = data['train_ids_partition']['current'][idx_minibatch]
        
    if idx_minibatch == 0:
        with open(settings.data_path + settings.dataset + '.csv', 'w') as f:
            f.write("target;prediction\n")
        print("Training 0 batch", len(train_ids_current_minibatch))
        # Batch training for first minibatch
        mf.fit(data, train_ids_current_minibatch, settings, param, cache)
    else:
        print('Evaluation on batch', idx_minibatch)
        results = mf.evaluate_predictions(data, data['x_train'][train_ids_current_minibatch], data['y_train'][train_ids_current_minibatch], settings, param, weights_prediction, True)
        # prediction
        predictions = results[0]['pred_mean']
        real = data['y_train'][train_ids_current_minibatch].flatten()
        for i in range(len(predictions)):
            print(i, predictions[i], real[i])
            with open(settings.data_path + settings.dataset + '.csv', 'a') as f:
                f.write("{0},{1}\n".format(real[i], predictions[i]))        
        
        print("Training on next batch ", idx_minibatch, len(train_ids_current_minibatch))
        # Online update
        mf.partial_fit(data, train_ids_current_minibatch, settings, param, cache)
    print("Finished training ...")
    # Evaluate
    weights_prediction = np.ones(settings.n_mondrians) * 1.0 / settings.n_mondrians
    train_ids_cumulative = data['train_ids_partition']['cumulative'][idx_minibatch]        
    
    
    

In [ ]:
train_ids_current_minibatch = data['train_ids_partition']['current'][5]

In [ ]:
results = mf.evaluate_predictions(data, data['x_train'][train_ids_current_minibatch], data['y_train'][train_ids_current_minibatch], settings, param, weights_prediction, True)

In [ ]:
predictions = results[0]['pred_mean']

In [ ]:
for i in range(len(predictions)):
    print(i, predictions[i], real[i])

In [ ]:
real = data['y_train'][train_ids_current_minibatch].flatten()

In [ ]:
data['x_train'][train_ids_current_minibatch]

In [ ]:
data['x_test']